In [1]:
import matplotlib.pyplot as plt
import time 
import numpy as np
from astropy.io import fits
import os 

In [2]:
directorio='/Users/ferre/Desktop/OSCURA/ImagenesOSCURA/'
contenido = os.listdir(directorio)

In [ ]:
##Prueba para verificar el algoritmo del demux
#No hace falta correr
for i in range(112):##se puede paralelizar este    
    for k in range (1200):
        img_total[:,k+nsamp*i]=scidata[:,(k*nsamp)+i]
        print("En img_total[",k +1200*i,"]"," cargo scidata[",(k*nsamp) + i,"]   k=",k,"    i=",i)

In [3]:
# Loop sobre todas las imagenes, se guarda en img_total cada ccd en fila demultiplexado
st = time.time()
#data=fits.HDUList([])
img_CCD16=fits.HDUList([]) #Se crearan nro_imagenes*nsamp/16 (Ej: 5*112/6=35)

for item in range(0,len(contenido)):
    hdulist = fits.open(directorio+contenido[item])
    tamx=int(hdulist[4].header['NAXIS1']) #134400
    tamy=int(hdulist[4].header['NAXIS2']) #Varia 
    nsamp=int(hdulist[4].header['NSAMP']) #112
    scidata = hdulist[4].data
    div=16 #Variable multiplo de 2^n 
    
    for j in range(0,int(nsamp/16)):                        #Se recorre nsamp/16=7 veces por cada imagen 
        img_parcial= np.zeros((tamy,int(div*(tamx/nsamp))),'i4')  #Se crean las imagenes parciales 
        for i in range(0,div):                               #Se recorre 16 veces para ir agarrando 16 CCD  
            for k in range(0,int(tamx/nsamp)):              #Se recorre tamx/nsamp=1200 veces para agarrar un CCD
                img_parcial[:,k+i*int(tamx/nsamp)]=scidata[:,(k*nsamp)+(i+div*j)] 
        img_CCD16.append(fits.ImageHDU(img_parcial))        #Se agrega una imagen a la lista tras recorrer 16 CCD
    #Proceso de guardado
    nombre=str("Imagen"+str(item)+".fits")
    hdul=fits.HDUList(img_CCD16)    
    hdul.writeto(nombre,overwrite=True)
    print('Se termino la imagen ',item)
    img_CCD16.clear()
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Se termino la imagen  0
Se termino la imagen  1
Se termino la imagen  2
Se termino la imagen  3
Se termino la imagen  4
Execution time: 16.726267337799072 seconds


In [116]:
type(img_CCD16[0])
print(img_CCD16[0].data.shape)

#plt.figure()
#plt.imshow(img_CCD16[:,0:7], cmap='gray')
#plt.colorbar()

(1300, 19200)


In [56]:
##Creando un archivo fits
#primary_hdu=fits.PrimaryHDU(img_CCD16[0])
#hdul=fits.HDUList(data)    
#hdul.writeto('new.fits',overwrite=True)
#print("Fin")

for j in range(0,len(img_CCD16)):        #Se analizan en grupos los nroimagenes*nsamp/div=5*112/16=35 archivos de 16 CCD c/u
    for i in range(0,int(nsamp/div)):    #Correspondiendo de 0-6(1),7-13(2)...
        for k in range(0,div):
            primary_hdu=fits.PrimaryHDU(img_CCD16[j].(:,0:int(div*(tamx/nsamp))))
            hdul=fits.HDUList(data)    
            hdul.writeto("New.Fits",i,".",j,overwrite=True)
    print("Fin de la imagen:",j)
        

TypeError: data object array(<astropy.io.fits.hdu.image.PrimaryHDU object at 0x00000216EB897E20>,
      dtype=object) should have at least one dimension

In [21]:
type(data[4])

astropy.io.fits.hdu.image.ImageHDU

In [ ]:
import concurrent.futures

In [ ]:
#Probar paralelizando
#Funcion a paralelizar demux
def demux(nombre):
    print("Comienza a demultiplexar \n")
    hdulist = fits.open(nombre)
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    img_total= np.zeros((tamy, tamx))
    tamxpimg=int(tamx/nsamp)
    
    for i in range(nsamp):
        for k in range (tamxpimg):
            img_total[:,k+tamxpimg*i]=scidata[:,(k*nsamp)+i]
    print('Se termino la imagen ') 
    

In [ ]:
type(hdulist)

In [ ]:
##Main
if __name__ == '__main__':    
    directorio='/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/'
    contenido = os.listdir(directorio)
    for j in range(0,len(contenido)):
        contenido[j]=directorio+contenido[j]
        
    t1 = time.perf_counter()
    #with concurrent.futures.ProcessPoolExecutor() as executor:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(demux, contenido)
    t2 = time.perf_counter()
    print(f'Finished in {t2-t1} seconds')


In [ ]:
from numba import njit,prange

In [ ]:
#Funcion a optimizar
#Necesita trabajar solo con arrays
@njit(parallel=True)
def demux2(tamx,tamy,nsamp,scidata):
    img_total= np.zeros((tamy, tamx))
    tamxpimg=int(tamx/nsamp)
    for i in prange(nsamp):##se puede paralelizar este
        
        for k in range (tamxpimg):
            img_total[:,k+tamxpimg*i]=scidata[:,(k*nsamp)+i]
            
    print('Se termino la imagen ')   


In [ ]:
directorio='/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/'
contenido = os.listdir(directorio) 
for j in range(0,len(contenido)):
    contenido[j]=directorio+contenido[j]
    
st = time.time()
for iterador in range(0,len(contenido)): #For para cada item del directorio
    hdulist = fits.open(contenido[iterador])
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    demux2(tamx,tamy,nsamp,scidata)
et = time.time()
elapsed_time = et - st
print('First Execution time :', elapsed_time, 'seconds')


st = time.time()
for iterador in range(0,len(contenido)): #For para cada item del directorio
    hdulist = fits.open(contenido[iterador])
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    demux2(tamx,tamy,nsamp,scidata)

et = time.time()
elapsed_time = et - st
print('Second Execution time :', elapsed_time, 'seconds')